https://womantalks.ru/forum/64-karera/

In [2]:
# pip install bs

In [3]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from tqdm.notebook import tqdm

df = pd.DataFrame(columns=['text'])
df

,text


In [4]:
url = 'https://womantalks.ru/forum/64-karera/'

In [5]:
response = requests.get(url)
print(response)

<Response [200]>


In [6]:
html = response.text

In [7]:
# bs = BeautifulSoup(response.text,"lxml")
# print(bs)

In [8]:
import re

In [9]:
def get_links(html, link):
    soup = BeautifulSoup(html, features="html.parser")
    links = []
    tds = soup.find_all('li', {'class':'ipsPagination_page'})
    for td in tds:
        p = td.find_all('a')
        for q in p:
            if link in q['href']:
                links.append(q['href'])
    return sorted(list(set(links)))
get_links(get_html(url), url)

[]

In [10]:
soup = BeautifulSoup(html, features="html.parser")

# quote
tds = soup.find_all('div', {'class':'ipsType_normal ipsType_richText ipsContained'})
temp = []
i = 0
for td in tds: 
    if td.find('blockquote') != None:
        p = td.find('div', {'class':"ipsQuote_contents"})
        if p is not None:
            temp.append(p.text)
    else:
        temp.append('')
print(temp)

[]


In [11]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import requests
import json

def get_html(url):
    response = requests.get(url)
    return response.text

def get_all_links(html):
    soup = BeautifulSoup(html, features="html.parser")
    links = []
    tds = soup.find_all('h4', {'class':'ipsDataItem_title ipsType_large ipsType_break'})
    for td in tds:
        p = td.find_all('a')
        for q in p:
            links.append(q['href'])
    return links

# def get_all_links_on_page(html, link):
#     soup = BeautifulSoup(html, features="html.parser")
#     tds = soup.find_all('tr')
#     for td in tds:
#         i = td.find('td', {'class':'td-block'})
#         if i != None:
#             return(i.find_all('a')[-1].text)
    
def get_links(html, link):
    soup = BeautifulSoup(html, features="html.parser")
    links = []
    tds = soup.find_all('li', {'class':'ipsPagination_page'})
    for td in tds:
        p = td.find_all('a')
        for q in p:
            if link in q['href']:
                links.append(q['href'])
    if len(links) == 0:
        return [link]
    return sorted(list(set(links)))

def get_all_links_one_theme(html):
    soup = BeautifulSoup(html, features="html.parser")
    links = []
    tds = soup.find_all('h4', {'class':'ipsDataItem_title ipsType_break'})
    for td in tds:
        p = td.find_all('a')
        for q in p:
            if 'topic' in q['href'] and \
            'https://womantalks.ru/topic/1512-kak-vyiuchit-razgovornyiy-angliyskiy/' not in q['href'] and \
            'https://womantalks.ru/topic/128-roman-na-rabote/' not in q['href']:
                links.append(q['href'])
    return links

def get_page_data(html):
    soup = BeautifulSoup(urlopen(str(html.encode('utf-8'))[2:-1]), features="html.parser")

    # user
    tds = soup.find_all('strong', 'itemprop'=="name")
    users = []
    for td in tds:
        if 'Изменено' not in td.text:
            users.append(td.text)
#     print(users)


    # time
    tds = soup.find_all('p', {'class':"ipsType_reset"})
    times = []
    for td in tds:
        if 'Опубликовано' in td.text:
            times.append(td.text)
#     print(times)


    # text
    tds = soup.find_all('div', {'class':'ipsType_normal ipsType_richText ipsContained'})
    tex = []
    for td in tds: 
        if td.find('blockquote') != None:
            p = td.find_all('p')
            tex.append(p[-1].text)
        else:
            tex.append(td.text)
#     print(tex)
    

    # quote
    tds = soup.find_all('div', {'class':'ipsType_normal ipsType_richText ipsContained'})
    temp = []
    for td in tds: 
        if td.find('blockquote') != None:
            p = td.find('div', {'class':"ipsQuote_contents"})
            if p is not None:
                temp.append(p.text)
        else:
            temp.append('')
#     print(temp)

    data = ({'user': users,                    
             'time': times,      
             'text': tex,            
             'quote': temp,          
             'topic': ['Job'] * len(users)
            })          
    return users, times, tex, temp, ['Job'] * len(users)#data

def page_content(all_links, table):
#     for link in tqdm(all_links):
#     links = get_links(get_html(all_links), all_links)
    for l in tqdm(all_links):
        all_links_one_theme = get_all_links_one_theme(get_html(l))
        for t in tqdm(all_links_one_theme):
#             print(t)
            page = get_links(get_html(t), t)
            for p in page:
#                 print(p)
                us, ti, te, tem, l = get_page_data(p)
                for i in range(len(us)):
                    if i > 5:
                        break
                    users.append(us[i])
                    times.append(ti[i])
                    tex.append(te[i])
                    temp.append(tem[i])
                    labels.append(l[i])
#                 table.append(data)

table = pd.DataFrame(columns={'users', 'times', 'text', 'quote', 'topic'})
users = []
times = []
tex = []
temp = []
labels = []
    
url = 'https://womantalks.ru/forum/64-karera/'

all_links = get_links(get_html(url), url)
print(all_links)
page_content(all_links, table)

table.loc[:, 'users'] = users
table.loc[:, 'times'] = times
table.loc[:, 'text'] = tex
table.loc[:, 'quote'] = temp
table.loc[:, 'topic'] = labels

['https://womantalks.ru/forum/64-karera/']


  0%|          | 0/1 [00:00<?, ?it/s]

0it [00:00, ?it/s]

In [20]:
pd.DataFrame(table).to_csv('Job_woman_talks_Copy.csv', index=False)

In [21]:
D = pd.read_csv('Job_woman_talks_Copy.csv')
D

,user,time,text,quote,topic
0,"['\nRebbekka', '\nСерсея', '\nRebbekka', '\nСе...","['\nОпубликовано: 12 ноя 2016\n', '\nОпубликов...","['\n\nДевочки, очень хочу научится делать перм...","['', '', '', '', '', '', '', '', '', '', '', '...","['Job', 'Job', 'Job', 'Job', 'Job', 'Job', 'Jo..."
1,"['\nAugustina', '\nAdelaida', '\nBrysnichka', ...","['\nОпубликовано: 14 авг 2018\n', '\nОпубликов...",['\n\n\tкласс) интересен ваш опыт) но опять же...,"['', '', '', '', '', '', '', '', '', '\n\n\t\t...","['Job', 'Job', 'Job', 'Job', 'Job', 'Job', 'Jo..."
2,"['\nblondi', '\nСофия0107', '\nKirra', '\nAugu...","['\nОпубликовано: 19 сен 2017\n', '\nОпубликов...","['\n\n\tДевчули, всем привет. Кто как зарабаты...","['', '', '', '', '', '\n\n\t\t\tСпособов зараб...","['Job', 'Job', 'Job', 'Job', 'Job', 'Job', 'Jo..."
3,"['\nИринка Матвеева', '\nIn-greed', '\nPolly',...","['\nОпубликовано: 7 фев 2016\n', '\nОпубликова...","['\n\n\tДавно хочу открыть какое-то свое дело,...","['', '', '', '', '', '', '', '', '', '', '', '...","['Job', 'Job', 'Job', 'Job', 'Job', 'Job', 'Jo..."
4,"['\nLana Voronina', '\nВероника В', '\nSveta81...","['\nОпубликовано: 18 апр 2017\n', '\nОпубликов...","['\n\n\tИдея, бизнес-план, стартовый капитал.\...","['', '', '', '', '\n\n\t\t\tИдея, бизнес-план,...","['Job', 'Job', 'Job', 'Job', 'Job', 'Job', 'Jo..."
5,"['\nikari', '\ncorenad', '\nlina.l']","['\nОпубликовано: 16 авг 2020\n', '\nОпубликов...",['\n\n\t\xa0Есть возможность инвестировать в г...,"['', '', '']","['Job', 'Job', 'Job']"
6,['\nТатьяна Мидорина'],['\nОпубликовано: 26 май 2020\n'],"['\n\n\tУважаемые девушки, всех приветствую ! ...",[''],['Job']
7,"['\nwomantalks.ru', '\nСчастье']","['\nОпубликовано: 7 апр 2020\n', '\nОпубликова...",['\n\n\tКто имеет опыт ведения бизнеса с близк...,"['', '']","['Job', 'Job']"
8,['\nmargo-c'],['\nОпубликовано: 4 мар 2020\n'],['\n\n\tВсем добрый день! Интересует работа в ...,[''],['Job']
9,"['\nАлин4ик', '\nHericussa', '\nsemenova', '\n...","['\nОпубликовано: 15 ноя 2018\n', '\nОпубликов...",['\n\n\tВсем привет! Может у кого есть оригина...,"['', '', '', '', '', '']","['Job', 'Job', 'Job', 'Job', 'Job', 'Job']"


In [22]:
count = 0
for i in range(len(D)):
    count += len(D['user'][i].split())
print('Количество записей с одного сайта по одной теме:', count)

Количество записей с одного сайта по одной теме: 382
